# Vertex enumeration

## Introduction to optimization and operations research

Michel Bierlaire


In [ ]:

from itertools import combinations

import numpy as np
import pandas as pd
from teaching_optimization.linear_constraints import (
    draw_polyhedron_standard_form,
    LabeledPoint,
)
from teaching_optimization.linear_optimization import LinearOptimization


In this lab, you will solve a small linear program by **vertex enumeration**.
You will list all candidate vertices by choosing basic indices, compute each **basic solution**,
test **feasibility**, and evaluate the **objective value** to identify the optimum.
This hands-on approach builds geometric intuition: vertices correspond to intersections of active
constraints, and the optimal solution must lie at one of these points for linear problems.
You will use the provided `LinearOptimization` class and `itertools.combinations` to automate the
enumeration, and plot the polyhedron (and level lines) to visualize why the best vertex is optimal.
Understanding this brute-force baseline will prepare you for more efficient methods like the **simplex**.

Consider the following optimization problem:
$$
\min_{x \in \mathbb{R}^2} 3x_1 + x_2
$$
subject to
\begin{align*}
2x_1+2x_2 & \geq 1,\\
x_1+x_2 & \leq 2,\\
x_1 & \geq 0,\\
x_2 & \geq 0.
\end{align*}

We want to find the optimal solution using vertex enumeration.
Hint: use the `LinearOptimization` class.

In [ ]:
help(LinearOptimization)


Here are examples of usage.

Constructor

In [ ]:
matrix_a = np.array([[1, 2, 3], [4, 5, 6]])
vector_b = np.array([7, 8])
vector_c = np.array([9, 10, 11])
example_linear_optimization = LinearOptimization(
    objective=vector_c, constraint_matrix=matrix_a, right_hand_side=vector_b
)
print(
    f'{example_linear_optimization.n_variables} variables, {example_linear_optimization.n_constraints} constraints.'
)


Specify the basic indices

In [ ]:
example_linear_optimization.basic_indices = [0, 1]


Obtain the basic solution

In [ ]:
print(example_linear_optimization.basic_solution)


Check if it is feasible.

In [ ]:
print(example_linear_optimization.is_basic_solution_feasible)


Obtain the value of the objective function at the vertex

In [ ]:
print(example_linear_optimization.value_objective_at_vertex)


You will also need the function `combinations` from `itertools`.

In [ ]:
help(combinations)


First, specify the data in standard form.

In [ ]:
standard_a = np.array([[-2, -2, 1, 0], [1, 1, 0, 1]])
standard_b = np.array([-1, 2])
the_objective = np.array([3, 1, 0, 0])


Create the LinearOptimization object

In [ ]:
the_linear_optimization = LinearOptimization(
    objective=the_objective, constraint_matrix=standard_a, right_hand_side=standard_b
)


Draw the polyhedron

In [ ]:
draw_polyhedron_standard_form(matrix_a=standard_a, vector_b=standard_b)


Loop over all feasible basic solutions and remember the best one.
For reporting, we store the output in a string matrix.

In [ ]:

best_solution = None
best_value = np.inf
report = []
headers = ['Basic indices', 'Vertex', 'Value', 'Feasible', 'Best value']


Loop over all possible combinations of basic indices.

In [ ]:
for basic_indices in combinations(
    range(the_linear_optimization.n_variables),
    the_linear_optimization.n_constraints,
):
    row = [f'{basic_indices}']
    the_linear_optimization.basic_indices = basic_indices
    if the_linear_optimization.basic_solution is None:
        row += ['Not a vertex']
    else:
        row += [f'{the_linear_optimization.basic_solution}']
    value: float = the_linear_optimization.value_objective_at_vertex
    row += [f'{value}']
    if the_linear_optimization.is_basic_solution_feasible:
        row += ['Yes']
        if value < best_value:
            best_value = value
            best_solution = the_linear_optimization.basic_solution
    else:
        row += ['No']
    row += [f'{best_value}']
    report.append(row)


We print the results using Pandas

In [ ]:
df = pd.DataFrame(report, columns=headers)
df


Optimal solution:

In [ ]:
print(f'Optimal solution: {best_solution}')


Optimal value:

In [ ]:
print(f'Optimal value: {best_value}')


Draw the optimal solution

In [ ]:
draw_polyhedron_standard_form(
    matrix_a=standard_a,
    vector_b=standard_b,
    objective_coefficients=the_objective,
    level_lines=[-0.5, 0.5, 1.5, 2.5],
    points=[LabeledPoint(coordinates=best_solution, label='Optimum')],
)